<a href="https://colab.research.google.com/github/cstecker/politicsRLab/blob/main/Wer_pr%C3%BCgelt_sich_mit_wem%3F_Statistische_Auswertung_von_Fistfights_in_Parlamenten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wer prügelt sich mit wem? Statistische Auswertung von *Fistfights* in Parlamenten

Manchmal sind Auseinandersetzungen innerhalb und ausserhalb von Debatten in Parlamenten ziemlich emotional. In einigen Fällen gehen Abgeordnete so weit, dass diese Auseinandersetzungen körperlich werden. Schauen wir uns ein Beispiel aus dem Kongress der Republik Guatemala an:

![](https://drive.google.com/uc?export=view&id=1RhSAQxjYqUoze-jqIBnG7MfrK1iiFB5d)

Hier geht es zum kompletten Video (spanisch-sprechende sind im Vorteil): https://www.youtube.com/watch?v=NGf8PS1RqYk

Es gibt viele dieser Vorkommnisse. Lasst uns einen weiteren Zwischenfall in der Jordanischen Abgeordnetenversammlung anschauen:

![](https://drive.google.com/uc?export=view&id=1QI2MrhxtZuj_jujHszlruxTxvSjb9oiV)

Hier geht es zum kompletten Video (arabisch-sprechende sind im Vorteil, achten Sie auch auf die Fangkünste des Abgeordneten bei [03:02](https://youtu.be/BgKIyTqLSPc?si=DwseB20oXZGxuLhW&t=182)): https://www.youtube.com/watch?v=BgKIyTqLSPc


Das sind nur zwei Beispiele von vielen weiteren. Nehmen Sie sich einmal zwei Minuten Zeit um über folgende Fragen nachzudenken:

1. In welchen Ländern mit welchen Eigenschaften findet physiche Gewalt in Parlamenten statt?
2. Welche Parlamentszusammensetzungen korrelieren mit solchen Ereignissen?
3. Zwischen welchen Gruppierungen findet Gewalt statt? (Opposition + Regierung; Regierung + Regierung; Opposition + Opposition)

Unter anderem diese Fragen haben sich die Politikwissenschaftler Moritz Schmoll und Wang Leung Ting systematisch in folgendem Paper angeschaut:

*Schmoll, Moritz; Ting, Wang Leung, 2024, "Fistfights In parliamentary Sessions Time Series (FISTS)", https://doi.org/10.7910/DVN/OBE9YJ, Harvard Dataverse, V3, UNF:6:1bE34iDnGirL6jIlDLyunw== [fileUNF]*

Den dort publizierten Datensatz bildet die Grundlage für die folgenden Analysen (und aus diesem stammen auch unsere beiden Beispiele von gerade eben).

In [1]:
# Wir laden einige Zusatzfunktionen in R
install.packages("plm")
library(tidyverse)
library(tibble)
library(httr)
library(plm)

# Laden unseres Datensatzes
fistfight_data <- haven::read_dta("https://github.com/cstecker/politicsRLab/blob/main/data/replication_data.dta?raw=true")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘rbibutils’, ‘bdsmatrix’, ‘collapse’, ‘zoo’, ‘sandwich’, ‘lmtest’, ‘maxLik’, ‘Rdpack’, ‘Formula’


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘plm’


The following objects are masked from ‘package:dplyr’:

    between, lag, lead




Lassen Sie uns kurz auf unseren Datensatz schauen:

In [2]:
# Basisinformationen zum Datensatz mit `glimpse()` laden
dplyr::glimpse(fistfight_data)

Rows: 8,229
Columns: 43
$ cname_year       <chr> "Afghanistan 1980", "Afghanistan 1981", "Afghanistan …
$ ccode            <dbl> 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,…
$ cname            <chr> "Afghanistan", "Afghanistan", "Afghanistan", "Afghani…
$ year             <dbl> 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988,…
$ ccodealp         <chr> "AFG", "AFG", "AFG", "AFG", "AFG", "AFG", "AFG", "AFG…
$ ccodealp_year    <chr> "AFG80", "AFG81", "AFG82", "AFG83", "AFG84", "AFG85",…
$ ccodecow         <dbl> 700, 700, 700, 700, 700, 700, 700, 700, 700, 700, 700…
$ ccodewb          <dbl> 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,…
$ dpi_maj          <dbl> NA, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, N…
$ vdem_polyarchy   <dbl> 0.08263069, 0.08288071, 0.08660699, 0.08636196, 0.086…
$ ucdp_type3       <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 4, 3, 3, 3, 4, 3, 1,…
$ wdi_wip          <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, 3.7, NA, NA, …
$ wdi_gdpcappppc

Uns stehen 8229 Zeilen mit 43 Spalten für unsere Analyse zur Verfügung. Was bedeuten die einzelnen Spalten und wie wurden sie codiert? Das alles (und mehr) finden Sie im offiziellen Codebook, welches Ihnen unter folgendem [Link](https://www.dropbox.com/scl/fi/tbv480ld4en3cqtzc75pr/FISTSv3.0-codebook.pdf?rlkey=1laui2jbnil9o3r2g3n6dfita&dl=1) zur Verfügung steht (Hinweis: wir arbeiten hier mit einer Kombination aus *2. Variables (raw data, .xlsx file)* und *3. Variables (dataset, .dta & .csv files)*). Werfen Sie einen Blick in das Codebook und beantworten Sie die folgenden Fragen:

1. Was ist der Observationszeitraum?
2. Wie wurde der Datensatz zusammengestellt? (Quellen, Suchbegriffe,...)
3. Welche Akteuere finden Beachtung in diesem Datensatz?

In [3]:
# Filtern auf unsere beiden Beispiele
fistfight_data_guatemala <- fistfight_data %>%
  filter(cname == "Guatemala", year == "2009")
View(fistfight_data_guatemala$h_lflo)
print(colnames(fistfight_data_guatemala))

[1] 0.8002903
attr(,"format.stata")
[1] "%10.0g"

 [1] "cname_year"       "ccode"            "cname"            "year"            
 [5] "ccodealp"         "ccodealp_year"    "ccodecow"         "ccodewb"         
 [9] "dpi_maj"          "vdem_polyarchy"   "ucdp_type3"       "wdi_wip"         
[13] "wdi_gdpcappppcur" "al_ethnic"        "ccp_syst"         "jw_election"     
[17] "demage"           "disprop"          "noclearmaj"       "reputational"    
[21] "poweraffecting"   "symbolic"         "simplepolicy"     "count"           
[25] "countb"           "lcount"           "lcountb"          "lndemage"        
[29] "yearb4election"   "important"        "dpi_system"       "dpi_yct"         
[33] "unimportant"      "h_l1"             "h_l2"             "h_lflo"          
[37] "time90"           "hum_trust"        "wdi_gini"         "sizeofmaj"       
[41] "abssizeofmaj"     "dpi_pr"           "dispropb"        


Schauen wir uns einmal das erste Beispiel aus Guatemala an. Die Ausgabe der Daten zu diesem Fall zeigt folgendes Muster: Der Faustkampf fand im Jahr 2009 statt. Zu diesem Zeitpunkt hatte Guatemala ein Demokratielevel (`vdem_polyarchy`) von 0.59 wobei 0 als sehr niedrig und 1 als sehr hoch gelten. Das Land befand sich nicht direkt vor einer Wahl (`yearb4election`) und es gab eine klare Mehrheit im Parlament (`noclearmaj`). Die Parteienvielfalt lag bei 0.8 (`h_lflo`). Schmoll und Ting (2024) schreiben zu dieser Metrik folgendes: "It takes on a value between 0 and 1 that denotes the probability of two parliamentarians, randomly drawn from the lower chamber of the parliament, representing different parties.".

Nun haben wir etwas anekdotisches Wissen über einen *Fistfight* in Guatemala (und ein Video zu Jordanien) gesammelt. Generalisierte Aussagen zu unseren Ausgangsfragen lassen sich so leider noch nicht wirklich treffen, da der Datensatz immerhin 8227 weitere Kampfsituationen enthält. Gute Nachrichten: Da wir mit R mit einer statistischen Programmiersprache arbeiten, müssen wir nicht all Observationen einzeln anschauen.

Im folgenden nutzen wir ein Regressionsmodell, mit welchem wir Korrelationen (also Zusammenhänge) zwischen Variablen erkennen können. Wir benötigen vorerst nur einen Befehl: `plm()`. In aller Regel rechnet man mehrere Modelle mit verschiedenen Spezifikationen, um robuste Zusammenhänge aufzeigen zu können. Jede Regresson benötigt eine abhängige Variable (=Depdendent Variable, DV oder Antwortvariable) welche in unserem Fall die Anzahl der Vorkommnisse pro Parlament und Landes-Jahr ist. Die unabhängigen Variablen (=Independent Variables, IV oder erklärende Variablen) variieren je nach Modell.

In [4]:
# Regressionsmodelle
# Filtern auf die Jahre nach 1990
fistfight_data_filtered <- fistfight_data %>%
  dplyr::filter(year >= 1990, h_l1 != 0) %>%
  pdata.frame(index = "year")

# Modell 2
m2 <- plm(count ~ vdem_polyarchy + c(vdem_polyarchy * vdem_polyarchy) + year, data = fistfight_data_filtered, model = "within", family = poisson)

# Modell 3
m3 <- plm(count ~ vdem_polyarchy + c(vdem_polyarchy * vdem_polyarchy) + abssizeofmaj + h_lflo, data = fistfight_data_filtered, model = "within", family = poisson)

In [5]:
summary(m2)

Oneway (individual) effect Within Model

Call:
plm(formula = count ~ vdem_polyarchy + c(vdem_polyarchy * vdem_polyarchy) + 
    year, data = fistfight_data_filtered, model = "within", family = poisson)

Unbalanced Panel: n = 27, T = 90-140, N = 3392

Residuals:
     Min.   1st Qu.    Median   3rd Qu.      Max. 
-0.168587 -0.106479 -0.078860 -0.040516  3.944692 

Coefficients:
                                   Estimate Std. Error t-value  Pr(>|t|)    
vdem_polyarchy                      0.63942    0.14012  4.5634 5.214e-06 ***
c(vdem_polyarchy * vdem_polyarchy) -0.57827    0.12155 -4.7573 2.044e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    404.72
Residual Sum of Squares: 401.97
R-Squared:      0.0067959
Adj. R-Squared: -0.0014735
F-statistic: 11.5054 on 2 and 3363 DF, p-value: 1.0478e-05

In [6]:
summary(m3)

Oneway (individual) effect Within Model

Call:
plm(formula = count ~ vdem_polyarchy + c(vdem_polyarchy * vdem_polyarchy) + 
    abssizeofmaj + h_lflo, data = fistfight_data_filtered, model = "within", 
    family = poisson)

Unbalanced Panel: n = 27, T = 87-133, N = 3233

Residuals:
     Min.   1st Qu.    Median   3rd Qu.      Max. 
-0.218602 -0.117970 -0.077886 -0.035084  3.912591 

Coefficients:
                                    Estimate Std. Error t-value  Pr(>|t|)    
vdem_polyarchy                      0.481979   0.152419  3.1622   0.00158 ** 
c(vdem_polyarchy * vdem_polyarchy) -0.511625   0.127967 -3.9981 6.529e-05 ***
abssizeofmaj                       -0.173266   0.054631 -3.1716   0.00153 ** 
h_lflo                              0.058599   0.036305  1.6141   0.10661    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    393.45
Residual Sum of Squares: 387.71
R-Squared:      0.014603
Adj. R-Squared: 0.0053709
F-statistic: 11.8631 on 4 

Was zeigen uns diese Ergebnisse? Zu allererst sehen wir die Stärke der Korrelation und das Signifikanzniveau der verschiedenen unabhänigen Variablen. Variablen ohne mindestens einen Stern (*) gelten als insignifikant nach dem 95%-Signifikanzniveau. Darüber hinaus gibt es Informationen zu den einzelnen Modellen: Wie gut können die unabhängigen Variablen die abhängige erklären? Das spiegelt sich im R^2-Wert wider. Modell 3 hat einen `Adj. R-Squared` von 0.0053709, wohingegen Modell 2 nur einen Wert von -0.0014735 aufweisen kann. Somit kann Modell 3 die Daten besser erklären als Modell 2.

Arbeiten wir nur mit dem Demokratielevel (Modell 2, `vdem_polyarchy`), finden wir einen signifikanten Koeffizienten. Ergänzen wir das Modell mit Marginality (`abssizeofmaj`) und dem Level der Parteienvielfalt (`h_lflo`), erkennen wir, dass zumindest Marginality einen signifikanten Effekt aufweist. Aber moment, was ist eigentlich Marginality? Die Autoren beschreiben die Variable in ihrem Artikel wie folgt: "Marginality is a measure of the balance of power between government and opposition parties within a parliament of a certain year."

Ab einer p-value (Spalte *Pr(>|t|)*) von *<0.05* erscheint ein Stern neben dem Koeffizienten (also befinden wir uns im 95%-Konfidenzintervall, ein Wert *<0.1* ergibt einen Punkt (das wäre das 90%-Konfidenzintervall). Unsere unabhängige Variable `h_lflo` in Modell 3 hat einen Wert von 0.10661, es wird also weder ein Stern noch ein Punkt angezeigt. Aber warum eigentlich dieses 95%-Konfidenzintervall? Reicht nicht auch 90%? Oder eben 89.339%? Theoretisch vielleicht schon, in der Praxis nicht. Auch wenn das 95%-Konfidenzintervall relativ willkürlich festgelegt wurde (und deshalb des Öfteren kritisiert wird), hat es sich an vielen Stellen in der Praxis durchgesetzt.

Geschafft! Wir sind von anekdotischem Wissen zu einer ersten Evidenz auf Basis unserere Daten gekommen.
Das war ein kleiner Exkurs in physische Gewalt in Parlamenten und der Analyse mit Hilfe von Regressionsmodellen in R. Sie haben nun zwei Möglichkeiten:

1.   Weitere Videos und Materialien zu *Fistfights* anschauen (siehe unten für den kompletten Datensatz mit Links zu den Quellen; achten Sie speziell auf die Spalte `URL`)
2.   Ihre eigenen Modelle auf einem anderen Datensatz laufen lassen

In [7]:
# Datensatz herunterladen -------------------------------------------------------------
download.file("https://github.com/cstecker/politicsRLab/blob/main/data/FISTSv3.0%20raw%20datafile%20(public).xlsx?raw=true", "FISTSv3.0 raw datafile (public).xlsx")
dataset_file <- "FISTSv3.0 raw datafile (public).xlsx"
# Datensatz einlesen -------------------------------------------------------------
fistfight_data_sources <- readxl::read_excel(dataset_file, 1L)
# Ausgabe des vollständigen Datensatz
# Tipp: Suchen Sie nach einzelnen Ländern oder Jahreszahlen? nutzen Sie folgenden Befehl, um mit Hilfe der zugehörigen Spalten nach diesen zu suchen:
# fistfight_data_sources %>%
#   filter(cname_year == "Ghana")
fistfight_data_sources

Country,Date,Maj. v. Opp.,Among Maj.,Among Opp.,No clear maj/opp,Reputational,Power-affecting,Symbolic,Simple policy,Comments,URL,Additional video source
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Afghanistan,37423,0,0,0,1,0,1,0,0,NA,https://www.nexis.com/docview/getDocForCuiReq?lni=4630-CXV0-00VK-H028&csi=280434&oc=00240&perma=true,.
Afghanistan,38845,0,0,0,1,1,0,0,0,NA,http://www.boston.com/news/world/asia/articles/2006/05/09/afghan_legislator_attacked_for_views/,.
Afghanistan,40577,.,.,.,.,.,.,.,.,No information from video.,https://www.youtube.com/watch?v=8CdHK-Pq9iA,.
Afghanistan,40730,0,0,0,1,1,0,0,0,NA,https://www.nexis.com/docview/getDocForCuiReq?lni=5382-XB01-JDJN-60D2&csi=280434&oc=00240&perma=true,.
Afghanistan,41611,0,0,0,1,1,0,0,0,NA,https://www.nexis.com/docview/getDocForCuiReq?lni=5B04-0V91-DYRV-30CF&csi=280434&oc=00240&perma=true,.
Afghanistan,42028,0,0,0,1,0,1,0,0,NA,https://www.nexis.com/results/enhdocview.do?docLinkInd=true&ersKey=23_T28907413348&format=GNBFI&startDocNo=1276&resultsUrlKey=0_T28907419220&backKey=20_T28907419221&csi=382497&docNo=1299,.
Afghanistan,42370,.,.,.,.,.,.,.,.,No information from video.,https://www.youtube.com/watch?v=HKGXejoR66Q,.
Afghanistan,42760,0,0,0,1,1,0,0,0,NA,https://www.nexis.com/docview/getDocForCuiReq?lni=5VMV-FDT1-F00C-60P2&csi=280434&oc=00240&perma=true,.
Afghanistan,43635,1,0,0,0,0,1,0,0,NA,https://www.rferl.org/a/angry-afghan-lawmakers-disrupt-parliament-over-disputed-vote/30010594.html,https://www.rferl.org/a/angry-afghan-lawmakers-disrupt-parliament-over-disputed-vote/30010594.html
